<a href="https://colab.research.google.com/github/midnight-koffee/neural_manifold/blob/main/Copy_of_haxbymanifold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install nilearn plotly umap-learn scikit-learn pandas numpy matplotlib seaborn
!pip install nibabel scipy

# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from nilearn import datasets, plotting, image
from nilearn.input_data import NiftiMasker
from nilearn.connectome import ConnectivityMeasure
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
# Download Haxby dataset (classic fMRI study)
haxby_dataset = datasets.fetch_haxby()
print("Dataset downloaded successfully.")
print(haxby_dataset)

# Load the functional data and labels
func_filename = haxby_dataset.func[0]  # Subject 1
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")

print("Functional data shape:", image.load_img(func_filename).shape)
print("Available conditions:", labels['labels'].unique())
print("Number of time points:", len(labels))

In [ ]:
from nilearn import datasets

# Tell nilearn to store in a specific folder
haxby_dataset = datasets.fetch_haxby(data_dir="C:/Users/debuc/nilearn_data")

print(haxby_dataset.func[0])         # Path to the functional MRI file
print(haxby_dataset.session_target)  # Path to labels file


In [ ]:
from nilearn import datasets

haxby_dataset = datasets.fetch_haxby()

print("Functional file path:", haxby_dataset.func[0])
print("Labels file path:", haxby_dataset.session_target[0])


In [ ]:
import pandas as pd
from nilearn import image

# Load the behavioral labels
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")

# Load the functional image and check its properties
func_img = image.load_img(haxby_dataset.func[0])

print("=== DATASET OVERVIEW ===")
print(f"Functional data shape: {func_img.shape}")
print(f"Available conditions: {labels['labels'].unique()}")
print(f"Number of time points: {len(labels)}")
print()

print("=== CONDITION COUNTS ===")
print(labels['labels'].value_counts())
print()

print("=== FIRST FEW ROWS OF LABELS ===")
print(labels.head(10))


In [ ]:
pip install nibabel nilearn matplotlib


In [ ]:
import os

# List everything in root
for root, dirs, files in os.walk('/content'):
    level = root.replace('/content', '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


In [ ]:

import nibabel as nib
from nilearn import plotting, image

# Set base path
base_path = "/content/C/haxby2001/subj2/"

# Load anatomical scan
anat = nib.load(base_path + "anat.nii.gz")
plotting.plot_anat(anat, title="Anatomical (anat.nii.gz)")

# Load functional fMRI time series
bold = nib.load(base_path + "bold.nii.gz")
print("Functional data shape:", bold.shape)  # (x, y, z, t)

# Show mean fMRI image (average across time)
mean_bold = image.mean_img(bold)
plotting.plot_epi(mean_bold, title="Mean Functional (bold.nii.gz)")

# Load and plot one of the masks
mask = nib.load(base_path + "mask4_vt.nii.gz")
plotting.plot_roi(mask, bg_img=anat, title="Mask overlayed on Anatomy")

# Show all plots
plotting.show()



In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np

# Load BOLD data (replace with your file path)
bold_img = nib.load("/content/C/haxby2001/subj2/bold.nii.gz")

# Convert to numpy array (shape: X × Y × Z × T)
bold = bold_img.get_fdata()

print("BOLD shape:", bold.shape)  # e.g., (64, 64, 40, 1452)

# Pick a slice in z (say slice 20 out of 40)
slice_idx = 20
bold_slice = bold[:, :, slice_idx, :]  # shape: (64, 64, 1452)

# Pick one voxel to see its time series
voxel_coords = (32, 32)  # center voxel
time_series = bold_slice[voxel_coords[0], voxel_coords[1], :]

# Plot voxel’s BOLD signal over time
plt.figure(figsize=(12, 5))
plt.plot(time_series)
plt.title(f"BOLD Time Series at Voxel {voxel_coords} (Slice {slice_idx})")
plt.xlabel("Time (TRs)")
plt.ylabel("Signal Intensity")
plt.show()

# Optional: visualize how the slice changes over time (first 100 frames)
plt.figure(figsize=(12, 12))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(bold_slice[:, :, i*10], cmap="gray")
    plt.title(f"Time {i*10}")
    plt.axis("off")

plt.suptitle(f"Slice {slice_idx} Evolution (every 10 TRs)")
plt.show()


In [ ]:
# BOLD Visualization and Animation - single cell
import os
import nibabel as nib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# ---- 1. Path to BOLD file ----
bold_file = "/content/C/haxby2001/subj2/bold.nii.gz"
if not os.path.exists(bold_file):
    raise FileNotFoundError(f"{bold_file} not found!")

# ---- 2. Load BOLD data ----
bold_img = nib.load(bold_file)
bold_data = bold_img.get_fdata()  # 4D: (x, y, z, time)
print("BOLD data shape:", bold_data.shape)

# ---- 3. Pick middle axial slice ----
slice_index = bold_data.shape[2] // 2
timepoints = bold_data.shape[3]

# ---- 4. Plot animation ----
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(bold_data[:, :, slice_index, 0], cmap='hot', origin='lower')
ax.set_title('BOLD Signal Evolution')

def update(frame):
    im.set_data(bold_data[:, :, slice_index, frame])
    return [im]

ani = FuncAnimation(fig, update, frames=timepoints, interval=100, blit=True)
plt.show()

# ---- 5. Optional: Save animation as MP4 ----
# Uncomment the next line if you want to save
# ani.save('/content/bold_animation.mp4', writer='ffmpeg', fps=10)


In [ ]:
# BOLD Slice Evolution Animation - single cell
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# ---- 1. Path to BOLD file ----
bold_file = "/content/C/haxby2001/subj2/bold.nii.gz"
if not os.path.exists(bold_file):
    raise FileNotFoundError(f"{bold_file} not found!")

# ---- 2. Load BOLD data ----
bold_img = nib.load(bold_file)
bold_data = bold_img.get_fdata()  # 4D: (x, y, z, time)
print("BOLD data shape:", bold_data.shape)

# ---- 3. Compute % signal change relative to voxel mean ----
mean_voxel = bold_data.mean(axis=3, keepdims=True)
psc_data = 100 * (bold_data - mean_voxel) / mean_voxel  # % signal change

# ---- 4. Select middle axial slice ----
slice_index = bold_data.shape[2] // 2
timepoints = bold_data.shape[3]

# ---- 5. Create animation ----
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(psc_data[:, :, slice_index, 0], cmap='hot', origin='lower', vmin=-5, vmax=5)
ax.set_title('BOLD % Signal Change Evolution')

def update(frame):
    im.set_data(psc_data[:, :, slice_index, frame])
    return [im]

ani = FuncAnimation(fig, update, frames=timepoints, interval=100, blit=True)
plt.show()

# ---- 6. Optional: Save animation as MP4 ----
# Uncomment the next line if you want to save the movie
ani.save('/content/bold_slice_evolution.mp4', writer='ffmpeg', fps=10)


In [ ]:
# BOLD Slice Evolution - Frame by Frame Visualization
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# ---- 1. Path to BOLD file ----
bold_file = "/content/C/haxby2001/subj2/bold.nii.gz"
if not os.path.exists(bold_file):
    raise FileNotFoundError(f"{bold_file} not found!")

# ---- 2. Load BOLD data ----
bold_img = nib.load(bold_file)
bold_data = bold_img.get_fdata()  # 4D: (x, y, z, time)
print("BOLD data shape:", bold_data.shape)

# ---- 3. Compute % signal change relative to voxel mean ----
mean_voxel = bold_data.mean(axis=3, keepdims=True)
psc_data = 100 * (bold_data - mean_voxel) / mean_voxel  # % signal change

# ---- 4. Select middle axial slice ----
slice_index = bold_data.shape[2] // 2
timepoints = bold_data.shape[3]
print(f"Visualizing slice {slice_index} across {timepoints} timepoints")

# ---- 5. Plot sequence of images ----
for t in range(timepoints):
    plt.figure(figsize=(5,5))
    plt.imshow(psc_data[:, :, slice_index, t], cmap='hot', origin='lower', vmin=-5, vmax=5)
    plt.title(f"BOLD % Signal Change - Timepoint {t+1}")
    plt.axis('off')
    plt.show()


In [ ]:
# BOLD Slice Animation - % Signal Change
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
from IPython.display import HTML

# ---- 1. Load BOLD data ----
bold_file = "/content/C/haxby2001/subj2/bold.nii.gz"
if not os.path.exists(bold_file):
    raise FileNotFoundError(f"{bold_file} not found!")

bold_img = nib.load(bold_file)
bold_data = bold_img.get_fdata()  # 4D: (x, y, z, time)

# ---- 2. Compute % signal change ----
mean_voxel = bold_data.mean(axis=3, keepdims=True)
psc_data = 100 * (bold_data - mean_voxel) / mean_voxel  # % signal change

# ---- 3. Middle axial slice ----
slice_index = bold_data.shape[2] // 2
timepoints = bold_data.shape[3]

# ---- 4. Create animation ----
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(psc_data[:, :, slice_index, 0], cmap='hot', origin='lower', vmin=-5, vmax=5)
ax.set_title('BOLD % Signal Change Evolution')
ax.axis('off')

def update(frame):
    im.set_data(psc_data[:, :, slice_index, frame])
    ax.set_title(f"BOLD % Signal Change - Timepoint {frame+1}")
    return [im]

ani = FuncAnimation(fig, update, frames=timepoints, interval=200, blit=True)  # 200ms per frame

# ---- 5. Display inline in Colab ----
HTML(ani.to_jshtml())

# ---- 6. Optional: Save animation ----
# ani.save('/content/bold_animation.mp4', writer=FFMpegWriter(fps=5))
# ani.save('/content/bold_animation.gif', writer=PillowWriter(fps=5))


In [ ]:
# Install required libraries (if not already installed)
!pip install nilearn nibabel matplotlib imageio

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import imageio
from nilearn import image as nli
from nilearn import plotting
import nibabel as nib
import os

# Load your BOLD data
bold_path = '/content/C/haxby2001/subj2/bold.nii.gz'
bold_img = nli.load_img(bold_path)

# Remove first few volumes to allow for signal stabilization (typically 4-5)
bold_img = bold_img.slicer[..., 4:]

# Get data dimensions
data = bold_img.get_fdata()
print(f"Data shape: {data.shape}")
n_x, n_y, n_z, n_volumes = data.shape

# Create mean image for background
mean_img = nli.mean_img(bold_img)

# Create a directory to store animation frames
!mkdir -p /content/animation_frames

# Create animation showing slice evolution over time
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title("BOLD Signal Evolution Over Time")

# Select a slice to visualize (middle slice)
slice_idx = n_z // 2

# Initialize the plot with the first volume
im = ax.imshow(data[:, :, slice_idx, 0], cmap='hot',
               origin='lower', aspect='auto')
fig.colorbar(im, ax=ax, label='BOLD Signal Intensity')

# Function to update animation frames
def update(frame):
    im.set_data(data[:, :, slice_idx, frame])
    ax.set_title(f"BOLD Signal Evolution - Volume {frame+1}/{n_volumes}")
    return [im]

# Create animation
ani = FuncAnimation(fig, update, frames=n_volumes,
                    interval=100, blit=True)

# Save as GIF
ani.save('/content/bold_evolution.gif', writer='pillow', fps=10)

# Close plot to avoid display issues
plt.close(fig)

# Display the animation in notebook
HTML('<img src="/content/bold_evolution.gif">')

In [ ]:
# Enhanced BOLD Slice Animation - fMRI style
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# ---- 1. Load BOLD data ----
bold_file = "/content/C/haxby2001/subj2/bold.nii.gz"
if not os.path.exists(bold_file):
    raise FileNotFoundError(f"{bold_file} not found!")

bold_img = nib.load(bold_file)
bold_data = bold_img.get_fdata()  # 4D: (x, y, z, time)
print("BOLD data shape:", bold_data.shape)

# ---- 2. Compute % signal change ----
mean_voxel = bold_data.mean(axis=3, keepdims=True)
psc_data = 100 * (bold_data - mean_voxel) / mean_voxel  # % signal change

# ---- 3. Create mean image as brain background ----
mean_img_data = bold_data.mean(axis=3)
slice_index = bold_data.shape[2] // 2
brain_slice = mean_img_data[:, :, slice_index]

# ---- 4. Reduce frames for faster, smoother animation ----
timepoints = bold_data.shape[3]
frames_to_use = range(0, timepoints, 5)  # every 5th frame

# ---- 5. Create figure ----
fig, ax = plt.subplots(figsize=(6,6))
ax.axis('off')

# Show brain background first
bg = ax.imshow(brain_slice, cmap='gray', origin='lower', alpha=0.6)

# Overlay BOLD % signal change on top
im = ax.imshow(psc_data[:, :, slice_index, 0], cmap='hot', origin='lower',
               vmin=-5, vmax=5, alpha=0.6)

cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('% BOLD Signal Change')

# ---- 6. Update function ----
def update(frame):
    im.set_data(psc_data[:, :, slice_index, frame])
    ax.set_title(f"BOLD Evolution - Volume {frame+1}", fontsize=12)
    return [im]

# ---- 7. Create animation ----
ani = FuncAnimation(fig, update, frames=frames_to_use, interval=100, blit=True)

# ---- 8. Display inline ----
HTML(ani.to_jshtml())

# ---- 9. Optional: Save MP4 ----
ani.save('/content/bold_slice_evolution_cool.mp4', writer='ffmpeg', fps=10)


In [ ]:
import pandas as pd
from nilearn import image

# Load the behavioral labels
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")

# Load the functional image and check its properties
func_img = image.load_img(haxby_dataset.func[0])

print("=== DATASET OVERVIEW ===")
print(f"Functional data shape: {func_img.shape}")
print(f"Available conditions: {labels['labels'].unique()}")
print(f"Number of time points: {len(labels)}")
print()

print("=== CONDITION COUNTS ===")
print(labels['labels'].value_counts())
print()

print("=== FIRST FEW ROWS OF LABELS ===")
print(labels.head(10))


In [ ]:
from nilearn.maskers import NiftiMasker

# Create a masker to extract voxel time series from functional images
masker = NiftiMasker(
    mask_strategy='background',  # automatically create brain mask
    standardize=True,           # z-score standardization
    detrend=True,               # remove linear trend
    low_pass=0.1,               # low-pass filtering at 0.1 Hz
    t_r=2.5                    # repetition time in seconds
)

print("Extracting neural activity from brain voxels...")
# This converts 4D fMRI data (x,y,z,time) into 2D matrix (time, voxels)
neural_activity = masker.fit_transform(func_img)

# Select only the stimulus conditions (exclude 'rest' and 'scrambledpix')
conditions_of_interest = ['face', 'house', 'cat', 'shoe', 'bottle', 'scissors']
condition_mask = labels['labels'].isin(conditions_of_interest)

# Filter to get only stimulus trials
X = neural_activity[condition_mask]
y = labels['labels'][condition_mask].values

print("=== NEURAL ACTIVITY EXTRACTION RESULTS ===")
print(f"Original 4D fMRI shape: {func_img.shape}")
print(f"Extracted 2D matrix shape: {neural_activity.shape}")
print(f"Selected conditions matrix: {X.shape}")
print(f"Selected conditions: {set(y)}")
print(f"Points per condition: {pd.Series(y).value_counts().iloc[0]}")


In [ ]:
# Install required packages for manifold analysis
!pip install umap-learn plotly

import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import plotly.express as px
import plotly.graph_objects as go

# Function to create different manifold embeddings
def create_manifold_embeddings(X, method='pca', n_components=2):
    """Create 2D manifold embeddings using different methods"""

    if method == 'pca':
        reducer = PCA(n_components=n_components, random_state=42)
    elif method == 'tsne':
        reducer = TSNE(n_components=n_components, random_state=42, perplexity=30)
    elif method == 'umap':
        reducer = umap.UMAP(n_components=n_components, random_state=42)

    print(f"Computing {method.upper()} embedding...")
    embedding = reducer.fit_transform(X)

    # Get variance explained if available (PCA only)
    if hasattr(reducer, 'explained_variance_ratio_'):
        variance_explained = reducer.explained_variance_ratio_
        print(f"Variance explained by first 2 components: {variance_explained[:2].sum():.3f}")
        return embedding, variance_explained
    else:
        return embedding, None

# Create embeddings with different methods
print("=== CREATING NEURAL MANIFOLDS ===")
methods = ['pca', 'tsne', 'umap']
embeddings = {}

for method in methods:
    emb, var_exp = create_manifold_embeddings(X, method=method)
    embeddings[method] = emb
    print(f"{method.upper()} embedding shape: {emb.shape}")
    print()

print("All manifold embeddings created successfully!")
print("Each point in these 2D spaces represents the brain state during one stimulus presentation")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create a comprehensive visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Define colors for each condition
unique_conditions = sorted(set(y))
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
color_map = {condition: colors[i] for i, condition in enumerate(unique_conditions)}

# Plot each manifold method
methods = ['pca', 'tsne', 'umap']
titles = ['PCA Manifold', 't-SNE Manifold', 'UMAP Manifold']

for i, (method, title) in enumerate(zip(methods, titles)):
    emb = embeddings[method]

    # Plot each condition with different colors
    for condition in unique_conditions:
        mask = y == condition
        axes[i].scatter(emb[mask, 0], emb[mask, 1],
                       c=color_map[condition], label=condition,
                       alpha=0.7, s=20)

    axes[i].set_title(title, fontsize=14)
    axes[i].set_xlabel('Component 1')
    axes[i].set_ylabel('Component 2')
    axes[i].grid(True, alpha=0.3)

    # Add legend only to first plot
    if i == 0:
        axes[i].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.suptitle('Neural Manifolds: Brain States During Visual Object Recognition',
             fontsize=16, y=1.02)
plt.show()

# Print summary statistics
print("=== MANIFOLD ANALYSIS SUMMARY ===")
print(f"Total brain states analyzed: {X.shape[0]}")
print(f"Brain voxels per state: {X.shape[1]:,}")
print(f"Stimulus conditions: {len(unique_conditions)}")
print(f"Trials per condition: {108}")
print(f"PCA variance explained (first 2 PCs): {var_exp[:2].sum():.1%}")


In [ ]:
import plotly.express as px
import pandas as pd

# Assume you already have:
# embeddings = {'pca': emb_pca, 'tsne': emb_tsne, 'umap': emb_umap}
# y = array of labels corresponding to each row in X

# Convert y to a pandas Series for convenience
y_series = pd.Series(y, name='Stimulus')

# Loop over embeddings and visualize
for method, emb in embeddings.items():
    # Create a DataFrame for plotting
    df_plot = pd.DataFrame({
        'Dim1': emb[:, 0],
        'Dim2': emb[:, 1],
        'Stimulus': y_series
    })

    # Interactive scatter plot
    fig = px.scatter(
        df_plot,
        x='Dim1',
        y='Dim2',
        color='Stimulus',
        title=f'Neural Manifold ({method.upper()})',
        width=800,
        height=600
    )

    fig.update_traces(marker=dict(size=6, opacity=0.7),
                      selector=dict(mode='markers'))
    fig.show()


In [ ]:
!git remote -v

In [ ]:
# 1. Clone your GitHub repo
!git clone https://github.com/midnight-koffee/neural_manifold.git
%cd neural_manifold

# 2. Copy your current notebook into the repo
!cp "/content/haxbymanifold.ipynb" .

# 3. Configure Git (if not done)
!git config --global user.email "debu.chatra@gmail.com"
!git config --global user.name "midnight-koffee"

# 4. Add, commit, and push
!git add .
!git commit -m "Add haxbymanifold notebook from Colab"
!git push origin main
